In [ ]:
import sys
!{sys.executable} -m pip install pandas requests openpyxl


In [8]:
import pandas as pd
import requests
import time

In [9]:
API_KEY = '97d7a79a3e538ab9f9237ca029f43cbe'

ALLOWED_TOWNSHIPS = {
    "闵行区": ["新虹街道", "华漕镇"],
    "长宁区": ["程家桥街道", "新泾镇"],
    "嘉定区": ["江桥镇"],
    "青浦区": ["徐泾镇"]
}

In [ ]:
input_file = "/Users/zoe/Desktop/矽亚资产管理/虹桥商务区/上海上市公司控股子公司名单-20250717-1.xlsx"  
df = pd.read_excel(input_file)

df["所属街道"] = ""
df["是否在目标区域"] = ""

print(df["控股企业办公地址"].head())

In [11]:
def get_location(address):
    url = f"https://restapi.amap.com/v3/geocode/geo?address={address}&key={API_KEY}"
    try:
        r = requests.get(url, timeout=5).json()
        if r.get("status") == "1" and r.get("geocodes"):
            return r["geocodes"][0]["location"]
    except Exception as e:
        print("地址解析失败:", e)
    return None

In [12]:
def get_township(location):
    url = f"https://restapi.amap.com/v3/geocode/regeo?location={location}&key={API_KEY}"
    try:
        r = requests.get(url, timeout=5).json()
        comp = r.get("regeocode", {}).get("addressComponent", {})
        return comp.get("district", ""), comp.get("township", "")
    except Exception as e:
        print("逆地理失败:", e)
    return "", ""

In [ ]:
for i, row in df.iterrows():
    addr = row["控股企业办公地址"]
    print(f"[{i+1}] 正在处理地址: {addr}")
    location = get_location(addr)
    if location:
        district, township = get_township(location)
        df.at[i, "所属街道"] = f"{district}{township}"

        if isinstance(district, str) and isinstance(township, str):
            if district in ALLOWED_TOWNSHIPS and township in ALLOWED_TOWNSHIPS[district]:
                df.at[i, "是否在目标区域"] = "是"
            else:
                df.at[i, "是否在目标区域"] = "否"
        else:
            df.at[i, "是否在目标区域"] = "否"
    else:
        df.at[i, "所属街道"] = "解析失败"
        df.at[i, "是否在目标区域"] = "否"
    time.sleep(0.2)  # 频率

In [15]:
df.to_excel("输出结果.xlsx", index=False)
print("完成")

完成
